In [56]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

df = spark.read.csv('data/airline_sentiment.csv', header=True)

In [57]:
df.toPandas().head()

,_unit_id,_golden,_unit_state,_trusted_judgments,_last_judgment_at,airline_sentiment,airline_sentiment:confidence,negativereason,negativereason:confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_id,tweet_location,user_timezone
0,681448150,FALSE,finalized,3,2/25/15 5:24,neutral,1,None,None,Virgin America,None,cairdin,None,0,@VirginAmerica What @dhepburn said.,None,2/24/15 11:35,5.70306E+17,None,Eastern Time (US & Canada)
1,681448153,FALSE,finalized,3,2/25/15 1:53,positive,0.3486,None,0,Virgin America,None,jnardino,None,0,@VirginAmerica plus you've added commercials t...,None,2/24/15 11:15,5.70301E+17,None,Pacific Time (US & Canada)
2,681448156,FALSE,finalized,3,2/25/15 10:01,neutral,0.6837,None,None,Virgin America,None,yvonnalynn,None,0,@VirginAmerica I didn't today... Must mean I n...,None,2/24/15 11:15,5.70301E+17,Lets Play,Central Time (US & Canada)
3,681448158,FALSE,finalized,3,2/25/15 3:05,negative,1,Bad Flight,0.7033,Virgin America,None,jnardino,None,0,"""@VirginAmerica it's really aggressive to blas...",None,2/24/15 11:15,5.70301E+17,None,Pacific Time (US & Canada)
4,681448159,FALSE,finalized,3,2/25/15 5:50,negative,1,Can't Tell,1,Virgin America,None,jnardino,None,0,@VirginAmerica and it's a really big bad thing...,None,2/24/15 11:14,5.70301E+17,None,Pacific Time (US & Canada)


In [58]:
df.toPandas()[['airline_sentiment', 'text']].head()

,airline_sentiment,text
0,neutral,@VirginAmerica What @dhepburn said.
1,positive,@VirginAmerica plus you've added commercials t...
2,neutral,@VirginAmerica I didn't today... Must mean I n...
3,negative,"""@VirginAmerica it's really aggressive to blas..."
4,negative,@VirginAmerica and it's a really big bad thing...


In [59]:
print((df.count(), len(df.columns)))

(14640, 20)


In [60]:
df = df.na.drop(subset=['text'])

In [61]:
print((df.count(), len(df.columns)))

(14638, 20)


In [62]:
from pyspark.ml.feature import Tokenizer

tokenizer = Tokenizer(inputCol='text', outputCol='tokenized')
df = tokenizer.transform(df)

In [63]:
from pyspark.ml.feature import CountVectorizer

cv = CountVectorizer(inputCol='tokenized', outputCol='features', minDF=5.0)

df = cv.fit(df).transform(df)

In [64]:
from pyspark.ml.feature import StringIndexer

indexer = StringIndexer(inputCol='airline_sentiment', outputCol='label')

df = indexer.fit(df).transform(df)

In [65]:
df_train, df_test = df.randomSplit([0.75, 0.25])

In [66]:
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

nb = NaiveBayes(smoothing=1.0, modelType='multinomial')

model = nb.fit(df_train)

In [67]:
df_predictions = model.transform(df_test)

In [68]:
evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(df_predictions)
print("Test set accuracy = " + str(accuracy))

Test set accuracy = 0.7813378302417088
